## Machine Translation
English to french

In [1]:
import collections

import numpy as np

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from tensorflow.keras.layers import Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import sparse_categorical_crossentropy

AlreadyExistsError: Another metric with the same name already exists.

### Verify access to the GPU

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14719032101380401830
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 2254123828
locality {
  bus_id: 1
  links {
  }
}
incarnation: 15774481516893467955
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1650 Ti, pci bus id: 0000:01:00.0, compute capability: 7.5"
]


## Dataset
[WMT](http://www.statmt.org/)

In [3]:
import os
def load_data(path):
    """
    Load dataset
    """
    input_file = os.path.join(path)
    with open(input_file, "r") as f:
        data = f.read()

    return data.split('\n')

In [4]:
# Load English data
english_sentences = load_data('small_vocab_en')
# Load French data
french_sentences = load_data('small_vocab_fr')

print('Dataset Loaded')

Dataset Loaded


### Files

In [5]:
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr Line 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
small_vocab_en Line 2:  the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr Line 2:  les Ã©tats-unis est gÃ©nÃ©ralement froid en juillet , et il gÃ¨le habituellement en novembre .


### Vocabulary

In [6]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


## Preprocess

### Tokenize (IMPLEMENTATION)

In [7]:
def tokenize(x):
    x_tk = Tokenizer(char_level = False)
    x_tk.fit_on_texts(x)
    return x_tk.texts_to_sequences(x), x_tk

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


### Padding (IMPLEMENTATION)

In [8]:
def pad(x, length=None):
    if length is None:
        length = max([len(sentence) for sentence in x])
    return pad_sequences(x, maxlen = length, padding = 'post')


# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


### Preprocess Pipeline

In [9]:
def preprocess(x, y):
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Data Preprocessed
Max English sentence length: 15
Max French sentence length: 21
English vocabulary size: 199
French vocabulary size: 345


In [10]:
preproc_english_sentences

array([[17, 23,  1, ..., 44,  0,  0],
       [ 5, 20, 21, ..., 51,  2, 45],
       [22,  1,  9, ..., 34,  0,  0],
       ...,
       [24,  1, 10, ..., 54,  0,  0],
       [ 5, 84,  1, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0]])

In [11]:
english_tokenizer

In [12]:
preproc_french_sentences

array([[[ 35],
        [ 34],
        [  1],
        ...,
        [  0],
        [  0],
        [  0]],

       [[  4],
        [ 32],
        [ 31],
        ...,
        [  0],
        [  0],
        [  0]],

       [[101],
        [  1],
        [ 12],
        ...,
        [  0],
        [  0],
        [  0]],

       ...,

       [[  7],
        [ 33],
        [  1],
        ...,
        [  0],
        [  0],
        [  0]],

       [[ 84],
        [  1],
        [ 20],
        ...,
        [  0],
        [  0],
        [  0]],

       [[  0],
        [  0],
        [  0],
        ...,
        [  0],
        [  0],
        [  0]]])

In [13]:
french_tokenizer

## Models

- Model 1 is a simple RNN
- Model 2 is a RNN with Embedding
- Model 3 is a Bidirectional RNN
- Model 4 is an optional Encoder-Decoder RNN

### Ids Back to Text
The neural network will be translating the input to words ids, which isn't the final form we want.  We want the French translation.  The function `logits_to_text` will bridge the gab between the logits from the neural network to the French translation.  You'll be using this function to better understand the output of the neural network.

In [14]:
def logits_to_text(logits, tokenizer):
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


### Model 1: RNN (IMPLEMENTATION)
A basic RNN model is a good baseline for sequence data.  In this model, you'll build a RNN that translates English to French.

In [15]:
def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    learning_rate = 1e-3
    input_seq = Input(input_shape[1:])
    rnn = GRU(64, return_sequences = True)(input_seq)
    logits = TimeDistributed(Dense(french_vocab_size))(rnn)
    model = Model(input_seq, Activation('softmax')(logits))
    model.compile(loss = sparse_categorical_crossentropy, 
                 optimizer = Adam(learning_rate), 
                 metrics = ['accuracy'])
    
    return model


In [16]:
# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

In [17]:
tmp_x.shape

(137861, 21, 1)

In [18]:
# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
simple_rnn_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 21, 1)]           0         
_________________________________________________________________
gru (GRU)                    (None, 21, 64)            12864     
_________________________________________________________________
time_distributed (TimeDistri (None, 21, 345)           22425     
_________________________________________________________________
activation (Activation)      (None, 21, 345)           0         
Total params: 35,289
Trainable params: 35,289
Non-trainable params: 0
_________________________________________________________________


In [19]:
simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

Epoch 1/10
108/108 [==============================] - 7s 25ms/step - loss: 3.4927 - accuracy: 0.4055 - val_loss: nan - val_accuracy: 0.4435
Epoch 2/10
108/108 [==============================] - 2s 18ms/step - loss: 2.5088 - accuracy: 0.4577 - val_loss: nan - val_accuracy: 0.4716
Epoch 3/10
108/108 [==============================] - 2s 18ms/step - loss: 2.2713 - accuracy: 0.4898 - val_loss: nan - val_accuracy: 0.5125
Epoch 4/10
108/108 [==============================] - 2s 18ms/step - loss: 2.0157 - accuracy: 0.5368 - val_loss: nan - val_accuracy: 0.5604
Epoch 5/10
108/108 [==============================] - 2s 18ms/step - loss: 1.8024 - accuracy: 0.5691 - val_loss: nan - val_accuracy: 0.5754
Epoch 6/10
108/108 [==============================] - 2s 18ms/step - loss: 1.6764 - accuracy: 0.5826 - val_loss: nan - val_accuracy: 0.5904
Epoch 7/10
108/108 [==============================] - 2s 18ms/step - loss: 1.5951 - accuracy: 0.5912 - val_loss: nan - val_accuracy: 0.5943
Epoch 8/10
108/108 [

In [20]:
# Print prediction(s)
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

paris jersey est parfois parfois en l' et il est est en en <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


### Model 2: Embedding (IMPLEMENTATION)

In [21]:
from tensorflow.keras.models import Sequential
def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    learning_rate = 1e-3
    rnn = GRU(64, return_sequences=True, activation="tanh")
    
    embedding = Embedding(french_vocab_size, 64, input_length=input_shape[1]) 
    logits = TimeDistributed(Dense(french_vocab_size, activation="softmax"))
    #TimeDistributed
    #This wrapper allows to apply a layer to every temporal slice of an input.
    #Every input should be at least 3D, and the dimension of index one of the first
    #input will be considered to be the temporal dimension.
    
    model = Sequential()
    #em can only be used in first layer --> Keras Documentation
    model.add(embedding)
    model.add(rnn)
    model.add(logits)
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model


# TODO: Reshape the input
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))

# TODO: Train the neural network

embeded_model = embed_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)

embeded_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)


# TODO: Print prediction(s)
print(logits_to_text(embeded_model.predict(tmp_x[:1])[0], french_tokenizer))

Epoch 1/10
108/108 [==============================] - 5s 28ms/step - loss: 3.7081 - accuracy: 0.4045 - val_loss: nan - val_accuracy: 0.4093
Epoch 2/10
108/108 [==============================] - 3s 25ms/step - loss: 2.6648 - accuracy: 0.4480 - val_loss: nan - val_accuracy: 0.5085
Epoch 3/10
108/108 [==============================] - 3s 24ms/step - loss: 2.0031 - accuracy: 0.5569 - val_loss: nan - val_accuracy: 0.5972
Epoch 4/10
108/108 [==============================] - 3s 24ms/step - loss: 1.5022 - accuracy: 0.6342 - val_loss: nan - val_accuracy: 0.6802
Epoch 5/10
108/108 [==============================] - 3s 24ms/step - loss: 1.1564 - accuracy: 0.7163 - val_loss: nan - val_accuracy: 0.7464
Epoch 6/10
108/108 [==============================] - 3s 25ms/step - loss: 0.9477 - accuracy: 0.7606 - val_loss: nan - val_accuracy: 0.7740
Epoch 7/10
108/108 [==============================] - 3s 25ms/step - loss: 0.8094 - accuracy: 0.7857 - val_loss: nan - val_accuracy: 0.7981
Epoch 8/10
108/108 [

### Model 3: Bidirectional RNNs (IMPLEMENTATION)

In [22]:
def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """

    learning_rate = 1e-3
    model = Sequential()
    model.add(Bidirectional(GRU(128, return_sequences = True, dropout = 0.1), 
                           input_shape = input_shape[1:]))
    model.add(TimeDistributed(Dense(french_vocab_size, activation = 'softmax')))
    model.compile(loss = sparse_categorical_crossentropy, 
                 optimizer = Adam(learning_rate), 
                 metrics = ['accuracy'])
    return model



tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

bidi_model = bd_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)


bidi_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=20, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(bidi_model.predict(tmp_x[:1])[0], french_tokenizer))

Epoch 1/20
108/108 [==============================] - 9s 45ms/step - loss: 2.6458 - accuracy: 0.4964 - val_loss: 1.8190 - val_accuracy: 0.5621
Epoch 2/20
108/108 [==============================] - 4s 39ms/step - loss: 1.6989 - accuracy: 0.5768 - val_loss: 1.5023 - val_accuracy: 0.6061
Epoch 3/20
108/108 [==============================] - 4s 39ms/step - loss: 1.4799 - accuracy: 0.6072 - val_loss: 1.3891 - val_accuracy: 0.6167
Epoch 4/20
108/108 [==============================] - 4s 39ms/step - loss: 1.3672 - accuracy: 0.6242 - val_loss: 1.3297 - val_accuracy: 0.6200
Epoch 5/20
108/108 [==============================] - 4s 39ms/step - loss: 1.2887 - accuracy: 0.6383 - val_loss: 1.2956 - val_accuracy: 0.6184
Epoch 6/20
108/108 [==============================] - 4s 40ms/step - loss: 1.2310 - accuracy: 0.6493 - val_loss: 1.2897 - val_accuracy: 0.6169
Epoch 7/20
108/108 [==============================] - 4s 40ms/step - loss: 1.1847 - accuracy: 0.6576 - val_loss: 1.3100 - val_accuracy: 0.6119

### Model 4: Encoder-Decoder 

In [23]:
def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """

    learning_rate = 1e-3
    model = Sequential()
    model.add(GRU(128, input_shape = input_shape[1:], return_sequences = False))
    model.add(RepeatVector(output_sequence_length))
    model.add(GRU(128, return_sequences = True))
    model.add(TimeDistributed(Dense(french_vocab_size, activation = 'softmax')))
    
    model.compile(loss = sparse_categorical_crossentropy, 
                 optimizer = Adam(learning_rate), 
                 metrics = ['accuracy'])
    return model


tmp_x = pad(preproc_english_sentences)
tmp_x = tmp_x.reshape((-1, preproc_english_sentences.shape[1], 1))

encodeco_model = encdec_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)

encodeco_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=20, validation_split=0.2)

print(logits_to_text(encodeco_model.predict(tmp_x[:1])[0], french_tokenizer))

Epoch 1/20
108/108 [==============================] - 8s 43ms/step - loss: 2.9813 - accuracy: 0.4430 - val_loss: 2.4524 - val_accuracy: 0.4955
Epoch 2/20
108/108 [==============================] - 4s 37ms/step - loss: 2.3107 - accuracy: 0.5001 - val_loss: 2.2085 - val_accuracy: 0.5067
Epoch 3/20
108/108 [==============================] - 4s 37ms/step - loss: 2.0856 - accuracy: 0.5163 - val_loss: 1.9217 - val_accuracy: 0.5253
Epoch 4/20
108/108 [==============================] - 4s 37ms/step - loss: 1.8331 - accuracy: 0.5477 - val_loss: 1.7507 - val_accuracy: 0.5597
Epoch 5/20
108/108 [==============================] - 4s 37ms/step - loss: 1.7005 - accuracy: 0.5677 - val_loss: 1.6518 - val_accuracy: 0.5773
Epoch 6/20
108/108 [==============================] - 4s 37ms/step - loss: 1.6159 - accuracy: 0.5773 - val_loss: 1.5780 - val_accuracy: 0.5849
Epoch 7/20
108/108 [==============================] - 4s 37ms/step - loss: 1.5466 - accuracy: 0.5872 - val_loss: 1.5179 - val_accuracy: 0.5885

### Model 5: Custom (IMPLEMENTATION)

In [24]:
def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    model = Sequential()
    model.add(Embedding(input_dim=english_vocab_size,output_dim=128,input_length=input_shape[1]))
    model.add(Bidirectional(GRU(256,return_sequences=False)))
    model.add(RepeatVector(output_sequence_length))
    model.add(Bidirectional(GRU(256,return_sequences=True)))
    model.add(TimeDistributed(Dense(french_vocab_size,activation='softmax')))
    learning_rate = 0.005
    
    model.compile(loss = sparse_categorical_crossentropy, 
                 optimizer = Adam(learning_rate), 
                 metrics = ['accuracy'])
    
    return model



print('Final Model Loaded')
# TODO: Train the final model

Final Model Loaded


## Prediction (IMPLEMENTATION)

In [25]:
len(english_tokenizer.word_index)+1, len(french_tokenizer.word_index)+1

(200, 346)

In [26]:
from tensorflow.keras.models import Sequential

"""
Gets predictions using the final model
:param x: Preprocessed English data
:param y: Preprocessed French data
:param x_tk: English tokenizer
:param y_tk: French tokenizer
"""

tmp_X = pad(preproc_english_sentences)
model = model_final(tmp_X.shape,
                    preproc_french_sentences.shape[1],
                    len(english_tokenizer.word_index)+1,
                    len(french_tokenizer.word_index)+1)

model.fit(tmp_X, preproc_french_sentences, batch_size = 1024, epochs = 17, validation_split = 0.2)


Epoch 1/17
108/108 [==============================] - 24s 169ms/step - loss: 2.0824 - accuracy: 0.5257 - val_loss: 1.3060 - val_accuracy: 0.6517
Epoch 2/17
108/108 [==============================] - 17s 158ms/step - loss: 1.0015 - accuracy: 0.7185 - val_loss: 0.7428 - val_accuracy: 0.7803
Epoch 3/17
108/108 [==============================] - 17s 158ms/step - loss: 0.5566 - accuracy: 0.8321 - val_loss: 0.4051 - val_accuracy: 0.8785
Epoch 4/17
108/108 [==============================] - 17s 159ms/step - loss: 0.2847 - accuracy: 0.9164 - val_loss: 0.2342 - val_accuracy: 0.9317
Epoch 5/17
108/108 [==============================] - 17s 159ms/step - loss: 0.1814 - accuracy: 0.9468 - val_loss: 0.1558 - val_accuracy: 0.9537
Epoch 6/17
108/108 [==============================] - 17s 159ms/step - loss: 0.1430 - accuracy: 0.9575 - val_loss: 0.1324 - val_accuracy: 0.9617
Epoch 7/17
108/108 [==============================] - 17s 159ms/step - loss: 0.1095 - accuracy: 0.9675 - val_loss: 0.1121 - val_ac

In [27]:
y_id_to_word = {value: key for key, value in french_tokenizer.word_index.items()}
y_id_to_word[0] = '<PAD>'
y_id_to_word

{1: 'est',
 2: 'en',
 3: 'il',
 4: 'les',
 5: 'mais',
 6: 'et',
 7: 'la',
 8: 'parfois',
 9: 'jamais',
 10: 'le',
 11: "l'",
 12: 'gã©nã©ralement',
 13: 'moins',
 14: 'aimã©',
 15: 'au',
 16: 'fruit',
 17: 'prã©fã©rã©',
 18: 'agrã©able',
 19: 'froid',
 20: 'son',
 21: 'chaud',
 22: 'de',
 23: 'plus',
 24: 'automne',
 25: 'mois',
 26: 'ã\xa0',
 27: 'elle',
 28: 'citrons',
 29: 'paris',
 30: 'inde',
 31: 'unis',
 32: 'ã©tats',
 33: 'france',
 34: 'jersey',
 35: 'new',
 36: 'chine',
 37: 'pendant',
 38: 'pamplemousse',
 39: 'mon',
 40: 'votre',
 41: 'juin',
 42: 'printemps',
 43: 'janvier',
 44: 'hiver',
 45: 'mars',
 46: 'ã©tã©',
 47: 'mai',
 48: 'septembre',
 49: 'juillet',
 50: 'avril',
 51: 'novembre',
 52: 'dã©cembre',
 53: 'fã©vrier',
 54: 'octobre',
 55: 'aime',
 56: 'aoã»t',
 57: 'merveilleux',
 58: 'relaxant',
 59: 'doux',
 60: 'humide',
 61: 'notre',
 62: 'californie',
 63: 'sec',
 64: 'leur',
 65: 'occupã©',
 66: 'pluvieux',
 67: 'calme',
 68: 'beau',
 69: 'habituellement',
 70

In [28]:
sentence = 'he saw a old yellow truck'
sentence = [english_tokenizer.word_index[word] for word in sentence.split()]
sentence

[26, 127, 100, 111, 112, 101]

In [29]:
sentence = pad_sequences([sentence], maxlen=preproc_english_sentences.shape[-1], padding='post')
sentence

array([[ 26, 127, 100, 111, 112, 101,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])

In [30]:
sentences = np.array([sentence[0], preproc_english_sentences[0]])
sentence

array([[ 26, 127, 100, 111, 112, 101,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])

In [31]:
predictions = model.predict(sentences, len(sentences))

In [32]:
print('Sample 1:')
print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
print('Il a vu un vieux camion jaune')

Sample 1:
il a vu un vieux camion jaune <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
Il a vu un vieux camion jaune


In [33]:
print('Sample 2:')
print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
print(' '.join([y_id_to_word[np.max(x)] for x in preproc_french_sentences[0]]))

Sample 2:
new jersey est parfois calme pendant l' automne et il est neigeux en avril <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
new jersey est parfois calme pendant l' automne et il est neigeux en avril <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


In [35]:
import pickle

# serialize model to JSON
model_json = model.to_json()
with open("translator_model.json", "w") as json_file:
    json_file.write(model_json)
    
# saving model
model.save('translator_model.h5')
print("Model saved")

# saving tokenizer
with open('english_tokenizer.pickle', 'wb') as handle:
    pickle.dump(english_tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# saving tokenizer
with open('french_tokenizer.pickle', 'wb') as handle:
    pickle.dump(french_tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
print('Tokenizer saved')

Model saved
Tokenizer saved
